In [1]:
import os
import h5py
import numpy as np
from IPython.lib.deepreload import reload
import matplotlib.pyplot as plt
from collections import OrderedDict
import skimage.io as io
%matplotlib qt

In [2]:
import xrdmaptools
from xrdmaptools.XRDMap import XRDMap
from xrdmaptools.reflections.SpotModels import GaussianFunctions
reload(xrdmaptools);

c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Connecting to databrokers...
failed.


In [3]:
scanid = 153126
#scanid = 153248
#scanid = 153485
filedir = 'D:\\Musterman_postdoc\\20240223_Musterman\\testing_maps\\'
#filedir = 'D:\\Musterman_postdoc\\20240223_Musterman\\xrd_maps\\'
h5_filename = f'scan{scanid}_xrd.h5'
test = XRDMap.from_hdf(h5_filename, wd=filedir, save_hdf=True, dask_enabled=False)

Loading data from hdf file...
Loading most recent images (final_images)...done!
Loading reciprocal positions...done!
Loading saved phases...done!
Loading scalers...done!
Loading positions...done!
Loading reflection spots...done!
Instantiating ImageMap...done!
XRD Map loaded!


In [4]:
filedir = 'D:\\Musterman_postdoc\\20240223_Musterman\\calibrations\\'

#test.set_calibration('scan153043_dexela_calibration.poni', filedir=filedir)
test.set_calibration('scan153220_dexela_calibration_ext.poni', filedir=filedir)

In [4]:
#dark_id = 153074
dark_id = 153086
dark_id = 153247
dark_id = 153481
dark_dir = 'D:\\Musterman_postdoc\\20240223_Musterman\\dark_fields\\'
dir_mask = [str(dark_id) in d for d in os.listdir(dark_dir)]

dark_field = io.imread(f'{dark_dir}{np.array(os.listdir(dark_dir))[dir_mask][0]}').astype(np.float32)
test.map.correct_dark_field(dark_field=dark_field)

Correcting dark-field...done!


In [5]:
#test.map.correct_outliers()
test.map.normalize_scaler()

Normalize image by i0 scaler...done!


In [6]:
test.map.apply_lorentz_correction(apply=False)
test.map.apply_polarization_correction()
test.map.apply_solidangle_correction()

Applying X-ray polarization correction...done!
Applying solid angle correction...done!


In [17]:
from xrdmaptools.utilities.utilities import delta_array
i = np.random.randint(test.map.map_shape[0])
j = np.random.randint(test.map.map_shape[1])

random_image = test.map.images[i, j].copy()
test.plot_image(random_image)

In [25]:
from xrdmaptools.utilities.utilities import delta_array
omega = 4 * np.arcsin(np.sin(np.radians(test.delta_tth / 2)) * np.sin(np.radians(test.delta_chi / 2)))

test.map.images /= (omega * np.sin(np.radians(test.tth_arr)))
test.map.corrections['lorentz'] = True

In [26]:
test.plot_image()

In [27]:
test.plot_interactive_map(img_vmin=0, img_vmax=1e-2)

In [28]:
#test.map.estimate_background(method='bruckner', binning=2, min_prominence=1)
test.map.estimate_background(method='bruckner', binning=2, min_prominence=0.1)

Estimating background with Bruckner algorithm.


100%|██████████| 6561/6561 [02:08<00:00, 51.07it/s]


In [30]:
test.plot_image(test.map.images[0, 1] - test.map.background[0, 1])

In [31]:
test.map.remove_background()

Removing background...done!


In [35]:
test.plot_image()

In [36]:
test.map.rescale_images(arr_min=0)
test.map.finalize_images()

Caution: Images not corrected for:
	absorption
	flat_field
	outliers
	pixel_defects
	pixel_distortions
	polar_calibration
Cleaning and updating image information...
Diffraction map size is 9.123 GB.
Compressing and writing images to disk.
This may take awhile...
done!


In [13]:
test.map.images[::2, ::2].shape

(11, 81, 972, 1536)

In [11]:
cols = np.arange(7, 150 + 1, 1)
rows = np.round(np.linspace(3, 16, len(cols)), 0).astype(np.int8)

In [12]:
image_vector = []

for col, row in zip(cols, rows):
    image_vector.append(test.map.images[row, col])

In [14]:
from xrdmaptools.utilities.image_corrections import rescale_array

save_dir = 'D:\\Musterman_postdoc\\20240223_Musterman\\plots\\xrd_maps\\'

for index, image in enumerate(image_vector):

    io.imsave(f'{save_dir}scan{test.scanid}_rotation_{index}.tiff',
              image,
              check_contrast=False,
              #plugin='pil'
              )

In [40]:
test.plot_image(image_vector[-1])

In [99]:
test.plot_interactive_map(img_vmin=0, img_vmax=0.1)

In [30]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
#test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="liNbO3 161")
#test.load_phase('Barker\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 148")
test.load_phase('Unknown\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3")
test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="platinum")
test.load_phase('AMCSD\\Calcite_0000984.cif', filedir=cif_dir, phase_name="calcite")
test.load_phase('AMCSD\\Stibnite_0008636.cif', filedir=cif_dir, phase_name="stibnite")
#test.load_phase('AMCSD\\Silicon_0011243.cif', filedir=cif_dir, phase_name="silicon")
#test.load_phase('AMCSD\\Corundum_0009327.cif', filedir=cif_dir, phase_name="sapphire")
#test.load_phase('AMCSD\\Hematite_0017806.cif', filedir=cif_dir, phase_name="hematite")
#test.load_phase('AMCSD\\Iron-alpha_0011214.cif', filedir=cif_dir, phase_name="iron-ferrite")
#test.load_phase('AMCSD\\Iron-beta_0011215.cif', filedir=cif_dir, phase_name="iron-beta")
#test.load_phase('AMCSD\\Iron-delta_0011216.cif', filedir=cif_dir, phase_name="iron-delta")
#test.load_phase('AMCSD\\Iron_0011146.cif', filedir=cif_dir, phase_name="iron-austenite")
#test.load_phase('AMCSD\\Cementite_0013523.cif', filedir=cif_dir, phase_name="cementite")
#test.load_phase('AMCSD\\Graphite_0011247.cif', filedir=cif_dir, phase_name="graphite")
#test.load_phase('AMCSD\\Tungsten_0011261.cif', filedir=cif_dir, phase_name="tungsten_1")
#test.load_phase('AMCSD\\Tungsten_0011236.cif', filedir=cif_dir, phase_name="tungsten_2")
#test.load_phase('AMCSD\\Iron_0011146.cif', filedir=cif_dir, phase_name="304SS-austenite")
#test.phases['304SS-austenite'].a = 3.529

In [31]:
test.select_phases(remove_less_than=0.1)

Updated phases saved in hdf.


In [11]:
from xrdmaptools.utilities.math import tth_2_d

tth_2_d(17.5, wavelength=test.wavelength) * np.sqrt(3)

3.529157094128613

In [58]:
from xrdmaptools.utilities.math import d_2_tth

for d, hkl in zip([2.075, 1.7961, 1.2697, 1.0828],
                  [[1, 1, 1], [2, 0, 0], [2, 2, 0], [3, 1, 1]]):
    print(d_2_tth(d, test.wavelength))
    h, k, l = hkl
    a = d * np.sqrt(h**2 + k**2 + l**2)
    #print(a)

17.181828968954655
19.8750430893204
28.259792849412545
33.26815679469825


In [12]:
test.plot_interactive_map(display_map=plot_map, img_vmin=0, img_vmax=0.1, map_vmin=0, map_vmax=200)

In [10]:
from xrdmaptools.crystal.Phase import phase_selector

tth, xrd = test.integrate_1d(image=np.max(test.map.images, axis=(0, 1)) / (omega * np.sin(np.radians(test.tth_arr))), tth_num=4096, unit='2th_deg')
phase_selector(xrd, list(test.phases.values()), tth, ignore_less=1)

{}

In [23]:
from xrdmaptools.utilities.image_corrections import rescale_array

fig, axes = plt.subplots(2, 2, figsize=(10, 5), dpi=200)
ax = axes.ravel()

div = [1, omega, np.sin(np.radians(test.tth_arr)), omega * np.sin(np.radians(test.tth_arr))]

for i in range(len(ax)):
    tth, xrd = test.integrate_1d(image=np.max(test.map.images, axis=(0, 1)) / div[i], tth_num=8192, unit='2th_deg')
    xrd = rescale_array(xrd, arr_min=0, upper=100)
    ax[i].plot(tth, xrd)

plt.show()

In [24]:
fig, axes = plt.subplots(2, 2, figsize=(10, 5), dpi=200)
ax = axes.ravel()

div = [1, omega, np.sin(np.radians(test.tth_arr)), omega * np.sin(np.radians(test.tth_arr))]

for i in range(len(ax)):
    tth, xrd = test.integrate_1d(image=test.map.images[0, 1] / div[i], tth_num=8192, unit='2th_deg')
    xrd = rescale_array(xrd, arr_min=0, upper=100)
    ax[i].plot(tth, xrd)

plt.show()

In [44]:
from xrdmaptools.reflections.spot_blob_search import spot_search

i = np.random.randint(test.map.map_shape[0])
j = np.random.randint(test.map.map_shape[1])

#out = spot_search(test.map.images[i, j], bkg_noise=map_noise, multiplier=3, plotme=True, mask=test.map.mask, expansion=0, sigma=3)
out = spot_search(test.map.images[i, j], multiplier=3, plotme=True,
                  mask=test.map.mask, expansion=10, size=3, threshold_method='minimum')

In [4]:
#test.map.images = rescale_array(test.map.images, lower=0, upper=100)
test.find_spots(multiplier=3, expansion=10, radius=10, size=3, threshold_method='minimum')

Searching images for spots...


100%|██████████| 6561/6561 [01:30<00:00, 72.41it/s]


Estimating spot characteristics...


100%|██████████| 16340/16340 [01:03<00:00, 255.49it/s]


Saving spots to hdf...done!


In [5]:
test.plot_image(spots=True, contours=True)

In [6]:
test.fit_spots(GaussianFunctions, max_dist=0.2, sigma=1)

Scheduling blob segmentation for spot fits...


100%|██████████| 6561/6561 [01:42<00:00, 63.73it/s]


Segmenting blobs for spot fits...


100%|██████████| 6561/6561 [02:53<00:00, 37.87it/s]


Fitting spots in blobs...


100%|██████████| 14122/14122 [05:51<00:00, 40.19it/s]

Successfully fit 16267 / 16340 spots ( 99.6 % ).
Saving spots to hdf...

done!


In [7]:
test.initial_spot_analysis()

Extracting more information from peak parameters...


100%|██████████| 16340/16340 [00:14<00:00, 1147.39it/s]

Converting peaks positions to q-space...done!
Saving spots to hdf...done!


In [9]:
test.trim_spots(remove_less=0.01)

Trimmed 0 spots less than 0.01 significance.


In [10]:
test.plot_image(spots=True, contours=True)

In [29]:
plot_map = np.zeros(test.map.map_shape)
plot_map[:] = np.nan

for image_index in range(test.map.num_images):
    indices = np.unravel_index(image_index, test.map.map_shape)
    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                          & (test.spots['map_y'] == indices[1])]
    
    if len(pixel_df) < 1:
        #plot_map[indices] = np.nan
        plot_map[indices] = 0
    else:
        #plot_map[indices] = np.sum(pixel_df['guess_int'])
        #plot_map[indices] = np.sum(pixel_df['fit_amp'])
        #plot_map[indices] = np.sum(pixel_df['fit_integrated'])
        #plot_map[indices] = np.mean(pixel_df['fit_fwhm_tth'])
        #plot_map[indices] = np.sum(pixel_df['fit_fwhm_chi'])
        #plot_map[indices] = np.mean(pixel_df['fit_chi0'])
        #plot_map[indices] = len(pixel_df)
        #plot_map[indices] = np.nanmedian(pixel_df['qy'])
        plot_map[indices] = np.mean(pixel_df['fit_r_squared'])
        #plot_map[indices] = pixel_df.iloc[np.argmax(pixel_df['fit_integrated'])]['fit_tth0']
        #plot_map[indices] = pixel_df.iloc[np.argmax(pixel_df['fit_integrated'])]['fit_chi0']
        pass


fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_map)
fig.colorbar(im, ax=ax)
ax.set_title('Spot Value')

plt.show()

In [435]:
test.plot_interactive_map(img_vmin=0, img_vmax=0.1)

In [13]:
test.plot_image(test.map.sum_map, aspect='equal')

In [24]:
from xrdmaptools.utilities.image_corrections import rescale_array

save_dir = 'D:\\Musterman_postdoc\\20240223_Musterman\\plots\\xrd_maps\\'

io.imsave(f'{save_dir}scan{test.scanid}_num_spots.tif', rescale_array(plot_map, upper=1, lower=0), check_contrast=False, plugin='pil')

In [18]:
from xrdmaptools.utilities.utilities import delta_array
from xrdmaptools.reflections.spot_blob_indexing import get_q_vect
from xrdmaptools.utilities.math import energy_2_wavelength

q_arr_naught = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

qx_min = q_arr_naught[0]
qx_max = q_arr_naught[0]
qy_min = q_arr_naught[1]
qy_max = q_arr_naught[1]
qz_min = q_arr_naught[2]
qz_max = q_arr_naught[2]

qx_delta = delta_array(q_arr_naught[0])
qy_delta = delta_array(q_arr_naught[1])
qz_delta = delta_array(q_arr_naught[2])

In [19]:
from xrdmaptools.utilities.utilities import delta_array
from xrdmaptools.reflections.spot_blob_indexing import get_q_vect
from xrdmaptools.utilities.math import energy_2_wavelength
from itertools import product

combos = list(product([0.002, 0, -0.002], [0.0016, 0, -0.0016], [0.0005, 0, -0.0005]))

for phi_x, phi_y, delta_E in combos:
    phi_z = 0

    lam = energy_2_wavelength(test.energy + delta_E)

    Rx = np.array([[1, 0, 0],
                   [0, np.cos(phi_x), -np.sin(phi_x)],
                   [0, np.sin(phi_x), np.cos(phi_x)]])

    Ry = np.array([[np.cos(phi_y), 0, np.sin(phi_y)],
                   [0, 1, 0],
                   [-np.sin(phi_y), 0, np.cos(phi_y)]])

    Rz = np.array([[np.cos(phi_z), -np.sin(phi_z), 0],
                   [np.sin(phi_z), np.cos(phi_z), 0],
                   [0, 0, 1]])

    R = Rz @ Ry @ Rz

    q_arr = get_q_vect(test.tth_arr, test.chi_arr, wavelength=lam)
    q_arr = (q_arr.T @ R).T


    qx_min = np.min([qx_min, q_arr[0]], axis=0)
    qx_max = np.max([qx_max, q_arr[0]], axis=0)
    qy_min = np.min([qy_min, q_arr[1]], axis=0)
    qy_max = np.max([qy_max, q_arr[1]], axis=0)
    qz_min = np.min([qz_min, q_arr[2]], axis=0)
    qz_max = np.max([qz_max, q_arr[2]], axis=0)

In [20]:
qx_min_ext = q_arr_naught[0] - qx_min
qx_max_ext = qx_max - q_arr_naught[0]
qy_min_ext = q_arr_naught[1] - qy_min
qy_max_ext = qy_max - q_arr_naught[1]
qz_min_ext = q_arr_naught[2] - qz_min
qz_max_ext = qz_max - q_arr_naught[2]

new_qx_delta = qx_delta + qx_min_ext + qx_max_ext
new_qy_delta = qy_delta + qy_min_ext + qy_max_ext
new_qz_delta = qz_delta + qz_min_ext + qz_max_ext

In [21]:
q_vol = new_qx_delta * new_qy_delta * new_qz_delta
q_vol_old = qx_delta * qy_delta * qz_delta
q_area = test.delta_tth * test.delta_chi

In [106]:
test.plot_image(q_vol)

In [105]:
test.plot_image(q_vol_old)

In [107]:
test.plot_image(q_area)

In [99]:
tth_arr = test.map.ai.twoThetaArray().astype(test.map.dtype)
chi_arr = test.map.ai.chiArray().astype(test.map.dtype)

# Check for discontinuities
if np.max(np.gradient(chi_arr)) > (np.pi / 6): # Semi-arbitrary cut off
    chi_arr[chi_arr < 0] += (2 * np.pi)

delta_chi = delta_array(chi_arr)
frac_chi = delta_chi / (2 * np.pi) # fraction of circle

# Relative Debye-Scherrer cone
#L3 = 1 / np.sin(tth_arr)
lc = frac_chi / np.sin(tth_arr)

In [109]:
test.plot_image(lc)

In [40]:
test.plot_image(indices, spots=True)

In [74]:
# Phase matching/mapping

#from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm

near_thresh = 0.075
sum_key = 'guess_int'
method = 'minimize_q_error'
method = 'maximize_spots'

phase_keys = list(test.phases.keys())
phase_list = list(test.phases.values())

[phase.get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)),
                               ignore_less=1)
        for phase in phase_list];

plot_map = np.zeros(test.map.map_shape)
plot_map[:] = 0

plot_dict = {}
for phase in phase_list:
    plot_dict[phase.name] = plot_map.copy()
plot_dict['unmatched'] = plot_map.copy()

for index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(index, test.map.map_shape)

    image = test.map.images[indices]
    pixel_df = test.pixel_spots(indices)
    #pixel_df = pixel_df.copy().dropna()
    num_spots = len(pixel_df)
    
    if num_spots > 1:
        spot_q_vals = np.linalg.norm(pixel_df[['qx', 'qy', 'qz']].values, axis=1)

        phase_diffs = []
        pot_matches = []
        phase_matches = []
        spot_matches = []
        for phase in phase_list:
            phase_q_vals = phase.reflections['q']
            #dist_arr = euclidean_distances(phase_q_vals.reshape(-1, 1), spot_q_vals.reshape(-1, 1))
            diff_arr = np.abs(phase_q_vals[:, np.newaxis] - spot_q_vals[np.newaxis, :])
            phase_diffs.append(diff_arr)
            pot_match = diff_arr < near_thresh
            pot_matches.append(pot_match)

            # Fraction of phase reflections explained by found spots
            #phase_matches.append(np.sum(pot_match / len(test.phases)))
            phase_matches.append(np.sum(np.any(pot_matches[-1], axis=1)) / len(phase_q_vals))

            # Number of spots explained by a given phase
            spot_matches.append(np.sum(np.any(pot_matches[-1], axis=0)))
            #raise

        # Track which spots have been matched
        rem_spot_mask = [True,] * num_spots

        if method in ['maximize_phase']:
            # Maximize phase matches
            # Should work better for polycrystalline patterns
            while (np.any(rem_spot_mask)) and (np.sum(phase_matches) > 0):
                # Check for more than 1 fully represented array
                if np.sum(np.array(phase_matches) == 1) > 1:
                    stacked_diffs = np.stack([np.min(diffs, axis=0) for diffs in phase_diffs])
                    spot_mask = np.min(stacked_diffs, axis=0) < near_thresh
                    phase_ind = np.argmin(stacked_diffs, axis=0)
                    phase_ind[~spot_mask] = -1
                    phase_mask = (np.array(phase_matches) == 1)
                    phase_count = [0,] * len(phase_list)
                    for ind in phase_ind:
                        if phase_mask[ind]:
                            phase_count[ind] += 1
                    max_ind = np.argmax(phase_count)
                else:
                    max_ind = np.argmax(phase_matches)

                # Index matching spots
                spot_mask = np.any(pot_matches[max_ind], axis=0)
                # Remove indexed spots from future use
                spot_mask *= rem_spot_mask
                rem_spot_mask *= ~spot_mask

                # Finally, find the value and remove reflections
                val = np.nansum(pixel_df[sum_key].values[spot_mask])
                phase_matches[max_ind] = 0

                # Fill value
                plot_dict[phase_list[max_ind].name][indices] = val

        elif method in ['maximize_spots']:
            # Maxmize spot_matches
            # Not sure when this one should work better
            while (np.any(rem_spot_mask)) and (np.sum(spot_matches) > 0):
                max_ind = np.argmax(spot_matches)
                # Index matching spots
                spot_mask = np.any(pot_matches[max_ind], axis=0)
                # Remove indexed spots from future use
                spot_mask *= rem_spot_mask
                rem_spot_mask *= ~spot_mask

                # Finally, find the value
                val = np.nansum(pixel_df[sum_key].values[spot_mask])
                spot_matches[max_ind] = 0

                # Fill value
                plot_dict[phase_list[max_ind].name][indices] = val

        elif method in ['minimize_q_error']:
        # Minimize Q difference
        # Should work better for perfect crystals and detector calibration...
            stacked_diffs = np.stack([np.min(diffs, axis=0) for diffs in phase_diffs])
            spot_mask = np.min(stacked_diffs, axis=0) < near_thresh
            phase_ind = np.argmin(stacked_diffs, axis=0)
            phase_ind[~spot_mask] = -1
            for i in range(len(phase_list)):
                val = np.nansum(pixel_df[sum_key].values[phase_ind == i])
                plot_dict[phase_list[i].name][indices] = val
            rem_spot_mask = ~spot_mask

        # Collect unmatched spots
        if np.any(rem_spot_mask):
            val = np.nansum(pixel_df[sum_key].values[rem_spot_mask])
            plot_dict['unmatched'][indices] = val


  0%|          | 0/19656 [00:00<?, ?it/s]

100%|██████████| 19656/19656 [00:32<00:00, 608.69it/s]


In [75]:
fig, axes = plt.subplots(2, 2, figsize=(10, 5), dpi=200)
ax = axes.ravel()

for i, key in enumerate(plot_dict.keys()):
    if key == 'unmatched':
        pass
        #continue

    im = ax[i].imshow(plot_dict[key], extent=extent)
    fig.colorbar(im, ax=ax[i], shrink=0.4)

    ax[i].set_title(key)

plt.tight_layout()

In [42]:
min_x = np.mean(test.pos_dict['map_x'][0])
max_x = np.mean(test.pos_dict['map_x'][-1])
min_y = np.mean(test.pos_dict['map_y'][:, 0])
max_y = np.mean(test.pos_dict['map_y'][:, -1])

min_x = np.mean(test.pos_dict['map_x'][:, 0])
max_x = np.mean(test.pos_dict['map_x'][:, -1])
min_y = np.mean(test.pos_dict['map_y'][0])
max_y = np.mean(test.pos_dict['map_y'][-1])

extent = [min_x, max_x, min_y, max_y]
#extent = [min_y, max_y, min_x, max_x]

In [71]:
from xrdmaptools.utilities.image_corrections import rescale_array

save_dir = 'D:\\Musterman_postdoc\\20240223_Musterman\\plots\\xrd_maps\\'

io.imsave(f'{save_dir}scan{test.scanid}_calcite_max_spots.tif', rescale_array(plot_dict['calcite'], upper=1, lower=0), check_contrast=False, plugin='pil')

In [37]:
from xrdmaptools.crystal.orientation import euler_rotation
from xrdmaptools.crystal.Phase import generate_reciprocal_lattice
from xrdmaptools.reflections.spot_blob_indexing import get_q_vect

skip = 500

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, -60, 0, 0)

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs/20)
#ax.scatter(*np.asarray(all_qs).T, c='k', s=all_fs/20)
ax.scatter(0, 0, 0, c='k', s=10)

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [845]:
are_coplanar([[-2, 3, -5], [-2, 3, -2], [-3, 4, -4], [-4, 5, -6]])

True

In [783]:
q1 = get_q_vect(test.tth_arr, test.chi_arr, wavelength=energy_2_wavelength(10.125))
q2 = get_q_vect(test.tth_arr, test.chi_arr, wavelength=energy_2_wavelength(22))

In [842]:
q1 = get_q_vect(test.tth_arr, test.chi_arr, wavelength=energy_2_wavelength(10.45))
#plot_qs = euler_rotation(all_qs, -60, 0, 0)
dist = euclidean_distances(plot_qs, q1.reshape(3, -1).T)
dist[dist == 0] = np.nan
indices = np.unravel_index(np.argmin(dist[dist > 0]), dist.shape)
min_dist = dist[indices]
min_hkl = all_hkls[indices[0]]
print(f'Closest reflection is the {min_hkl} at {min_dist:.4f} Å⁻¹ from the detector.')

Closest reflection is the (-2, 3, -5) at 0.3401 Å⁻¹ from the detector.


In [758]:
max_depth = 0

for q1_i, q2_i in tqdm(zip(q1.reshape(3, -1).T, q2.reshape(3, -1).T)):
    dist = euclidean_distances(q1_i.reshape(-1, 1).T, q2_i.reshape(-1, 1).T)[0, 0]
    max_depth = np.max([dist, max_depth])

373248it [01:14, 5004.75it/s]


In [747]:
q_range = list(q1.reshape(3, -1).T) + list(q2.reshape(3, -1).T)

In [19]:
all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['sapphire'], tth_range=(0, 60))
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, -60, 0, 0)
ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs/20)
#ax.scatter(*np.asarray(all_qs).T, c='k', s=all_fs/20)
ax.scatter(0, 0, 0, c='k', s=10)

ax.scatter(*np.asarray(plot_qs)[13], c='r', s=10)

for theta in [0, 5, 5.5, 12.25, 13.625, 14.20, 15.55]:
    theta = np.radians(theta)

    q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=energy_2_wavelength(18))

    Ry = np.array([[np.cos(theta), 0, -np.sin(theta)],
                   [0, 1, 0],
                   [np.sin(theta), 0, np.cos(theta)]])

    # Plot sampled Ewald sphere
    q_mask = q[:, test.map.mask]

    q_mask = (q_mask.T @ Ry).T
    ax.plot_trisurf(q_mask[0].ravel()[::skip],
                    q_mask[1].ravel()[::skip],
                    q_mask[2].ravel()[::skip],
                    alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [700]:
are_coplanar([[1, 0, 0], [2, 0, 0], [1, 1, 0]])

True

In [270]:
pixel_indices = [0, 0]
skip = 500

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, 10, -20, 0)

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.scatter(qx, qy, qz, c='b', s=1, alpha=0.1)

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [23]:
pixel_indices = [0, 0] #[26, 21]
skip = 500

#fig, ax = plt.subplots(1, 1, figsize=(5, 10), dpi=200, subplot_kw={'projection':'3d'})
fig = plt.figure(figsize=(10, 5), dpi=200)
ax = fig.add_axes([0, 0, 0.5, 1], projection='3d')
#ax = fig.add_subplot(projection='3d')

plot_qs = euler_rotation(all_qs, 10, -20, 0)
recip_latt = ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

phi1, PHI, phi2 = 10, -20, 0
euler_angles = {'phi1' : phi1,
                'PHI' : PHI,
                'phi2': phi2}
euler_bounds = [[-180, 180], [0, 180], [-180, 180]]
slider_lst = []
update_lst = []

slider_vpos = np.linspace(0.8, 0.1, 3)

for i, key in enumerate(euler_angles.keys()):
    slider_ax = fig.add_axes

    # Make a horizontal slider to control the frequency.
    axfreq = fig.add_axes([0.7, slider_vpos[i], 0.2, 0.03])
    euler_slider = Slider(
        ax=axfreq,
        label=f'{key} [deg]',
        valmin=euler_bounds[i][0],
        valmax=euler_bounds[i][1],
        valinit=euler_angles[key],
    )

    slider_lst.append(euler_slider)

    # The function to be called anytime a slider's value changes
    def update_factory(key):
        def update(val):
            global recip_latt
            euler_angles[key] = val
            plot_qs = euler_rotation(all_qs, *euler_angles.values())
            recip_latt.remove()
            recip_latt = ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
            fig.canvas.draw_idle()
        return update

    update_lst.append(update_factory(key))
    slider_lst[i].on_changed(update_lst[i])

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [158]:
test.spots = pd.read_hdf(test.hdf_path, key='xrdmap/reflections/spots')

In [159]:
dropped_indices = []

for index in test.spots.index:
    spot = test.spots.loc[index]
    if spot['fit_amp'] - spot['fit_offset'] < 0.5:
        dropped_indices.append(index)

In [160]:
test.spots = test.spots.drop(index = dropped_indices)

In [132]:
from xrdmaptools.crystal.orientation import iterative_dictionary_indexing

pixel_indices = 2, 29
pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                        & (test.spots['map_y'] == pixel_indices[1])].copy()
spot_qs = pixel_df[['qx', 'qy', 'qz']].values

fit_ori, fit_min = iterative_dictionary_indexing(spot_qs, test.phases['stibnite'], [np.min(test.tth_arr), np.max(test.tth_arr)],
                                                 cut_off=0.05,
                                                 start_angle=20,
                                                 angle_resolution=0.1, 
                                                 euler_bounds=[[0, 180], [0, 90], [0, 180]])

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

In [50]:
skip = 500

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, *fit_ori[-2])

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [527]:
test.plot_image([2, 28], spots=True)

In [65]:
# Informed dictionary indexing

from scipy.spatial.transform import Rotation
from sklearn.metrics.pairwise import euclidean_distances
from xrdmaptools.crystal.orientation import iterative_dictionary_indexing, euler_rotation

indices = 11, 17

phase = test.phases['stibnite']
near_thresh = 0.01
tth_range = [np.min(test.tth_arr), np.max(test.tth_arr)]
cut_off = 0.05
start_angle = 20
angle_resolution = 0.1 
euler_bounds = [[0, 180], [0, 90], [0, 180]]

# Get useful information out of pixel_df
# Does not condense spots...
pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
pixel_df.dropna(inplace=True)

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))
all_qs = np.asarray(all_qs)

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

# Conditionals to avoid single spot from one grain
if len(pixel_df) < 2:
    raise ValueError()

dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values)
if np.max(dist) < min_q:
    raise ValueError()


spot_qs = pixel_df[['qx', 'qy', 'qz']].values
spot_ints = pixel_df[['guess_int']].values

# find potential matces within a threshold
phase_q_vals = phase.reflections['q']
spot_q_vals = np.linalg.norm(spot_qs, axis=1)
diff_arr = np.abs(spot_q_vals[:, np.newaxis] - phase_q_vals[np.newaxis, :])
pot_match = diff_arr < near_thresh
iso_spots = np.sum(pot_match, axis=1) == 1

#iso_spots = -1

# Find rotation matrix just good enough for indexing
# Explicit solution
if np.sum(iso_spots) > 1:
    print('Explicit solution...')
    #max_iso = np.max(spot_ints[iso_spots])
    #spot_ind = np.squeeze(spot_ints == max_iso)
    q_meas = spot_qs[iso_spots]
    #q_calc = phase.Q(phase.reflections['hkl'][pot_match[iso_spots][0]])
    q_calc = phase.Q([phase.reflections['hkl'][ind][0] for ind in pot_match[iso_spots]])
    # Fit in q-space
    rot, rssd = Rotation.align_vectors(q_meas, q_calc)

    if rssd > 1:
        print(f'Explicit fitting exceeded error threshold ({rssd:.3f}). Downgrading method...')
        max_iso = np.max(spot_ints[iso_spots])
        iso_spots = np.squeeze(spot_ints == max_iso)
    else:
        best_orientation = rot.as_matrix()

# Iterate around a single axis of rotation
if np.sum(iso_spots) == 1:
    print('Dictionary indexing around a reflection...')
    q_meas = spot_qs[iso_spots]
    print(f"Rotation center at {pixel_df['guess_img_x'][iso_spots].values[0]}, {pixel_df['guess_img_y'][iso_spots].values[0]}")
    #q_calc = phase.Q(phase.reflections['hkl'][pot_match[iso_spots][0]])
    q_calc = phase.Q([phase.reflections['hkl'][ind][0] for ind in pot_match[iso_spots]])
    # Fit in q-space
    rot_init, rssd = Rotation.align_vectors(q_calc, q_meas)
    norm_q = np.squeeze(q_meas / np.linalg.norm(q_meas))

    ITERATE = True
    # The upper angle may be set by the large symmetric rotation of the point group
    # The step is reduced since there are few iterations for the linear problem 
    angles = np.arange(0, 180, start_angle / 4)
    step = start_angle
    while ITERATE:
        step /= 2 # half the resolution each time
        if step <= angle_resolution:
            ITERATE = False

        # extra rotation about initial rotation
        ext_rot = Rotation.from_rotvec(norm_q[np.newaxis, :]* angles[:, np.newaxis],
                               degrees=True).as_matrix()
        #tot_rot = rot_init.as_matrix() @ ext_rot
        tot_rot = ext_rot @ rot_init.as_matrix()

        min_list = []
        for R_mat in tot_rot:
            dist = euclidean_distances(spot_qs, all_qs @ R_mat)
            min_list.append(np.sum(np.min(dist, axis=1)**2))
        
        min_mask = min_list < cut_off * (np.max(min_list) - np.min(min_list)) + np.min(min_list)
        best_angles = np.asarray(angles)[min_mask]
        best_orientation = tot_rot[np.argmin(min_list)] # this is the output!

        new_angles = []
        for angle in best_angles:
            new_angles.append(angle - step)
            new_angles.append(angle)
            new_angles.append(angle + step)

        angles = np.unique(new_angles)
    
    if np.min(min_list) > 0.5:
        print(f'Axial rotation fitting exceeded error threshold ({np.min(min_list):.3f}). Downgrading method...')
        iso_spots = -1

# No clue. Iterate around mesh of orientations
if np.sum(iso_spots) < 1:
    print('Full iterative dictionary indexing...')
    fit_ori, fit_min = iterative_dictionary_indexing(spot_qs,
                                                     phase,
                                                     tth_range,
                                                     cut_off=cut_off,
                                                     start_angle=start_angle,
                                                     angle_resolution=angle_resolution, 
                                                     euler_bounds=euler_bounds)
    
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    best_orientation = euler_rotation(I, *fit_ori[-1])
    print(f'Completed with ({fit_min[-1]:.3f}).')


# find next largest iso_spot and try fit explicit rotation matrix...

# generate rotations about meas_hkl vector...


# refine rotation matrix based on indexing values...

Explicit solution...
Explicit fitting exceeded error threshold (1.580). Downgrading method...
Dictionary indexing around a reflection...
Rotation center at 427, 644
Axial rotation fitting exceeded error threshold (0.996). Downgrading method...
Full iterative dictionary indexing...
Completed with (0.152).


In [66]:
best_orientation

array([[-0.82135298,  0.02364123,  0.56993015],
       [-0.42730007, -0.68739137, -0.58728848],
       [ 0.37788084, -0.72590234,  0.57469285]])

In [84]:
skip = 500
pixel_indices = indices

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = all_qs @ best_orientation

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()

if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [8]:
test.phases['stibnite'].get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

In [225]:
pot_match.shape

(259, 403)

In [241]:
# Explicit indexing

from scipy.spatial.transform import Rotation
from sklearn.metrics.pairwise import euclidean_distances
from xrdmaptools.crystal.orientation import iterative_dictionary_indexing, euler_rotation
from xrdmaptools.crystal.Phase import generate_reciprocal_lattice
from xrdmaptools.reflections.spot_blob_indexing import get_q_vect
from itertools import product
from tqdm import tqdm

indices = 3, 158

phase = test.phases['stibnite']
near_thresh = 0.05
required_spots = 2

# Get useful information out of pixel_df
# Does not condense spots...
pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
pixel_df.dropna(inplace=True)

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))
all_qs = np.asarray(all_qs)

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

# Conditionals to avoid single spot from one grain
if len(pixel_df) < 2:
    raise ValueError()

dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values[:required_spots])
if np.max(dist) < min_q:
    raise ValueError()


spot_qs = pixel_df[['qx', 'qy', 'qz']].values
spot_ints = pixel_df[['guess_int']].values

# Sort spots from largest to smallest intensity
zipped = list(zip(spot_ints, spot_qs))
zipped.sort(reverse=True)
spot_ints, spot_qs = zip(*zipped)


# find potential matces within a threshold
phase_q_vals = phase.reflections['q']
spot_q_vals = np.linalg.norm(spot_qs, axis=1)
diff_arr = np.abs(spot_q_vals[:, np.newaxis] - phase_q_vals[np.newaxis, :])
pot_match = diff_arr < near_thresh

phase_mask = np.any(pot_match, axis=1)
pot_match = pot_match[phase_mask]
spot_qs = np.array(spot_qs)[phase_mask]
#spot_int = spot_ints[phase_mask]


fit_q_meas = spot_qs[:required_spots]
fit_q_calc = []
for i, pot_match_i in enumerate(pot_match[:required_spots]):
    match_ind = np.where(pot_match_i)[0]
    hkls = [phase.reflections['hkl'][ind] for ind in match_ind]

    # Add equivalent potential indices. Only first index restricted
    # Not sure how much this actually helps...
    if i > 0:
        ext_hkls = []
        [ext_hkls.extend(list(phase.lattice.equivalent_hkls(hkl))) for hkl in hkls];
        hkls = ext_hkls

    q_calc = phase.Q(hkls)
    fit_q_calc.append(q_calc)

combos = list(product(*fit_q_calc))

rot_list, rssd_list = [], []
for q_calc in tqdm(combos):
    rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
    rot_list.append(rot.as_matrix())
    rssd_list.append(rssd)

min_ind = np.argmin(rssd_list)
best_orientation = rot_list[min_ind]
print(f'Best orientation found at {min_ind} with rssd of {rssd_list[min_ind]:.3f}')

for combo in combos[min_ind][1:]:
    if phase.HKL(combo) not in phase.reflections['hkl']:
        print('Equivalent hkl used successfully. The extra computation was worth it!')
        break

ValueError: 

In [238]:
def explicit_indexing(spot_qs, # Must already be sorted...
                      phase,
                      near_thresh=0.05,
                      required_spots=2,
                      check_equivalent=True):

    # Sort spots from largest to smallest intensity
    #zipped = list(zip(spot_ints, spot_qs))
    #zipped.sort(reverse=True)
    #spot_ints, spot_qs = zip(*zipped)

    # find potential matces within a threshold
    phase_q_vals = phase.reflections['q']
    spot_q_vals = np.linalg.norm(spot_qs, axis=1)
    diff_arr = np.abs(spot_q_vals[:, np.newaxis] - phase_q_vals[np.newaxis, :])
    pot_match = diff_arr < near_thresh

    # Distinguish by phase kind of
    phase_mask = np.any(pot_match, axis=1)
    pot_match = pot_match[phase_mask]
    spot_qs = np.array(spot_qs)[phase_mask]
    #spot_int = spot_ints[phase_mask]

    fit_q_meas = spot_qs[:required_spots]
    fit_q_calc = []
    for i, pot_match_i in enumerate(pot_match[:required_spots]):
        match_ind = np.where(pot_match_i)[0]
        hkls = [phase.reflections['hkl'][ind] for ind in match_ind]

        # Add equivalent potential indices. Only first index restricted
        # Not sure how much this actually helps...
        if check_equivalent and i > 0:
            ext_hkls = []
            [ext_hkls.extend(list(phase.lattice.equivalent_hkls(hkl))) for hkl in hkls];
            hkls = ext_hkls

        q_calc = phase.Q(hkls)
        fit_q_calc.append(q_calc)

    combos = list(product(*fit_q_calc))

    rot_list, rssd_list = [], []
    for q_calc in combos:
        rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
        rot_list.append(rot.as_matrix())
        rssd_list.append(rssd)

    min_ind = np.argmin(rssd_list)
    best_orientation = rot_list[min_ind]
    #print(f'Best orientation found at {min_ind} with rssd of {rssd_list[min_ind]:.3f}')

    # This was worth it!
    #for combo in combos[min_ind][1:]:
    #    if phase.HKL(combo) not in phase.reflections['hkl']:
    #        print('Equivalent hkl used successfully. The extra computation was worth it!')
    #        break

    return best_orientation, rssd_list[min_ind]

In [131]:
skip = 500
pixel_indices = indices

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

#plot_qs = all_qs @ rot_list[np.argmin(rssd_list)]
plot_qs = all_qs @ best_orientation

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()

if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [152]:
I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

euler_mat = euler_rotation(I, 10, 10, 10)
Rotation.from_matrix(euler_mat).as_euler('zxz', degrees=True)

array([10., 10., 10.])

In [641]:
# Explicit indexing across full map

from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm

required_spots = 3
phase = test.phases['stibnite']

if phase.reflections is None:
    phase.get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

euler_map = np.empty((*test.map.map_shape, 3))
fit_map = np.empty(test.map.map_shape)

for index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(index, test.map.map_shape)
    phi1, PHI, phi2 = np.nan, np.nan, np.nan

    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
    pixel_df.dropna(inplace=True)
    
    # Conditional to avoid too few spots for theoretical indexing
    if len(pixel_df) < required_spots:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    spot_qs = pixel_df[['qx', 'qy', 'qz']].values
    spot_ints = pixel_df[['fit_integrated']].values
    #spot_chis = pixel_df[['fit_chi0']].values

    spot_qs, spot_ints = combine_nearby_spots(spot_qs, spot_ints, max_dist=0.1, max_neighbors=np.inf)

    # Sort spots from largest to smallest intensity
    zipped = list(zip(spot_ints, spot_qs))
    zipped.sort(reverse=True)
    spot_ints, spot_qs = zip(*zipped)
    spot_ints = list(spot_ints)
    spot_qs = list(spot_qs)

    dist_too_small = True
    while dist_too_small:
        if len(spot_qs) < required_spots:
            euler_map[indices] = phi1, PHI, phi2
            dist_too_small = False
            break
        # Conditional to avoid single spot from one blob
        dist = euclidean_distances(spot_qs)
        
        if dist[0, 1] < min_q:
            del spot_qs[1], spot_ints[1]
        else:
            dist_too_small = False

    if len(spot_qs) < required_spots:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    best_orientation, min_rssd = explicit_indexing(spot_qs,
                                                   phase,
                                                   near_thresh=0.05,
                                                   required_spots=required_spots,
                                                   check_equivalent=True)
    
    best_euler = Rotation.from_matrix(best_orientation).as_euler('zxz', degrees=True)
    
    euler_map[indices] = best_euler
    fit_map[indices] = min_rssd

 15%|█▍        | 504/3381 [00:31<12:03,  3.98it/s] C:\Users\emusterma\AppData\Local\Temp\ipykernel_26832\1301288915.py:44: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
 36%|███▌      | 1207/3381 [03:42<32:46,  1.11it/s]  C:\Users\emusterma\AppData\Local\Temp\ipykernel_26832\1301288915.py:44: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
 40%|████      | 1356/3381 [04:44<12:55,  2.61it/s]  C:\Users\emusterma\AppData\Local\Temp\ipykernel_26832\1301288915.py:44: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
 40%|████      | 1367/3381 [04:55<46:34,  1.39s/it]C:\Users\emusterma\AppData\Local\Temp\ipykernel_26832\1301288915.py:44: UserWarning: Optimal rotation is not uniquely or

KeyboardInterrupt: 

In [7]:
# Iterative dictionary indexing across full map

from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm
from xrdmaptools.reflections.spot_blob_indexing import iterative_dictionary_indexing
from xrdmaptools.crystal.Phase import generate_reciprocal_lattice

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

euler_map = np.empty((*test.map.map_shape, 3))
fit_map = np.empty(test.map.map_shape)

for index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(index, test.map.map_shape)
    phi1, PHI, phi2 = np.nan, np.nan, np.nan

    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
    pixel_df.dropna(inplace=True)
    spot_qs = pixel_df[['qx', 'qy', 'qz']].values

    # Conditional to avoid too few spots for theoretical indexing
    if len(pixel_df) < 3:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    # Conditional to avoid single spot from one grain
    dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values)
    if np.max(dist) < min_q:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    fit_ori, fit_min = iterative_dictionary_indexing(spot_qs, test.phases['stibnite'], [np.min(test.tth_arr), np.max(test.tth_arr)],
                                                 cut_off=0.01,
                                                 start_angle=20,
                                                 angle_resolution=1, 
                                                 euler_bounds=[[0, 180], [0, 90], [0, 180]])
    
    euler_map[indices] = fit_ori[-1]
    fit_map[indices] = fit_min[-1]

 14%|█▍        | 482/3381 [02:49<40:42,  1.19it/s]  

In [638]:
from xrdmaptools.crystal.orientation import g_func
g = g_func(*euler_map.reshape(test.map.num_images, 3).T)

In [639]:
ipole = [0, 1, 0]

colors, r, theta = [], [], []
for i in range(len(g)):
    
    # Take only part of the orientation matrix
    V = np.dot(g[i], ipole) # order is g, then ipole

    # Determine pixel color according to unit triangle
    R, G, B = 0, 0, 0
    base = 30 / 255 #all pixels are a bit brighter
    R=np.abs(V[2]) + base # referenced to crystallographic axes, but should be able to reference any axis
    G=np.abs(V[0]) + base
    B=np.abs(V[1]) + base

    # Normalize and brighten colors
    max_c = np.max([R,G,B])
    R = R / max_c
    G = G / max_c
    B = B / max_c
    
    # Setting pixel transparency if called
    A=1
    colors.append([R, G, B, A])

colors = np.asarray(colors)
colors = colors.reshape((*test.map.map_shape, 4))

In [640]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

ax.imshow(colors)
ax.set_title(str(ipole))

plt.show()

In [340]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

fit_map[fit_map < 0.0001] = np.nan

ax.imshow(fit_map)
ax.set_title('Fit')

plt.show()

In [341]:
norm_fit_map = fit_map.copy()

for index in range(test.map.num_images):
    indices = np.unravel_index(index, test.map.map_shape)
    
    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
    pixel_df.dropna(inplace=True)

    # Conditional to avoid too few spots for theoretical indexing
    if len(pixel_df) < 3:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    # Conditional to avoid single spot from one grain
    dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values)
    if np.max(dist) < min_q:
        euler_map[indices] = phi1, PHI, phi2
        continue

    norm_fit_map[indices] /= len(pixel_df)

fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

im = ax.imshow(norm_fit_map)
fig.colorbar(im, ax=ax)

plt.show()

In [343]:
from xrdmaptools.utilities.image_corrections import rescale_array

In [344]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

trans_colors = colors.copy()
trans_colors[:, :, -1] = -rescale_array(norm_fit_map, lower=-1, upper=0)
mask = trans_colors[:, :, -1] == colors[:, :, -1]
trans_colors[mask, -1] = 0

ax.imshow(trans_colors)
ax.set_title(str(ipole))
ax.set_facecolor('black')

plt.show()

In [346]:
test.plot_interactive_map(display_map=colors, img_vmin=0, img_vmax=0.1)

In [599]:
#from xrdmaptools.utilities.utilities import label_nearest_spots
from xrdmaptools.geometry.geometry import *

indices = 9, 75

pixel_df = test.pixel_spots(indices).copy()
pixel_df.dropna(inplace=True)
q_spots = pixel_df[['qx', 'qy', 'qz']].values
polar_spots = pixel_df[['fit_tth0', 'fit_chi0']].values
spot_ints = pixel_df[['guess_int']].values


out = label_nearest_spots(q_spots, max_dist=0.1, max_neighbors=np.inf)
new_out = np.empty((len(out), 3))
new_out[:, -1] = out[:, -1]
new_out[:, :2] = q_2_polar(out, wavelength=test.wavelength)
out = new_out

In [633]:
from xrdmaptools.utilities.utilities import arbitrary_center_of_mass

def combine_nearby_spots(spots, *weights, max_dist, max_neighbors=np.inf):
    # Spots are weighted by the first eight!
    
    labeled_spots = label_nearest_spots(spots, max_dist=max_dist, max_neighbors=max_neighbors)

    combined_spots = []
    combined_weights = []
    for label in np.unique(labeled_spots[:, -1]):
        label_mask = labeled_spots[:, -1] == label
        #combined_spot = np.mean(labeled_spots[label_mask][:, :-1], axis=0)
        combined_spot = arbitrary_center_of_mass(np.squeeze(np.asarray(weights)[0])[label_mask],
                                                  *labeled_spots[label_mask][:, :-1].T)
        combined_spots.append(combined_spot)
        for weight in weights:
            combined_weight = np.sum(weight[label_mask])
            combined_weights.append(combined_weight)

    return combined_spots, combined_weights

In [634]:
out, weights = combine_nearby_spots(q_spots, spot_ints, max_dist=0.1, max_neighbors=np.inf)
new_out = np.empty((len(out), 3))
new_out[:, :2] = q_2_polar(out, wavelength=test.wavelength)
out = new_out

In [635]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.images[indices], vmin=0, vmax=0.1)
fig.colorbar(im, ax=ax)

ax.scatter(*estimate_image_coords(out[:, :-1], test.tth_arr, test.chi_arr).T,
           #c=out[:, -1],
           c=weights,
           s=1,
           cmap='jet')

plt.show()

In [ ]:
elements_folder = f'output_tiff_scan2D_{scanid}_xs_sum8ch\\'

element_maps = {}
for element in os.listdir(f'{filedir}{elements_folder}'):
    el_im = io.imread(f'{filedir}{elements_folder}{element}')
    element_maps[str(element[:-5])] = el_im

In [ ]:
key = 'detsum_Fe_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(test.map.images, range(test.map.images.shape[0]), range(test.map.images.shape[1]), display_map=element_maps[key], display_title=key)